In [2]:
import torch.nn as nn

In [3]:
class edge_coder(nn.Module):
    def __init__(self,in_dim,hidden_dim,out_dim,edge_drop):
        super(edge_coder,self).__init__()
        self.l1=nn.Linear(in_dim,hidden_dim)
        self.l2=nn.Linear(hidden_dim,out_dim)
        self.l3=nn.Linear(hidden_dim,out_dim)
        nn.init.xavier_normal_(self.l1.weight,gain=1.414)
    def forward(self,shuru):
        out=self.l1(shuru)
        out=F.relu(out)
      #  out=F.relu(self.l2(out))
    #    out=F.relu(self.l3(out))
        #    out=self.l2(out)
        return out

In [4]:
class GCN1(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN1, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)
        self.l1=nn.Linear(in_feats,out_feats)
        self.l2=nn.Linear(in_feats,out_feats)
    def message_func(self,edges):
        return {'h':edges.src['h'],'a_drop':edges.data['a_drop'],'aft':edges.data['aft']}
        
    def reduce_func(self,nodes):
        #alpha=F.softmax(nodes.mailbox['e'],dim=1)
        alpha=nodes.mailbox['a_drop']
  #      aft=nodes.mailbox['aft']
     #   h=nodes.mailbox['h']
       # aft=aft.view(-1,64)     
      #  print(alpha.shape)
  #      print(h.shape)
  #      print(aft.shape)
        z=torch.sum(alpha*nodes.mailbox['h']+F.leaky_relu(self.l1(nodes.mailbox['aft'])),dim=1)
     #   z=F.leaky_relu(self.l2(z))
     #   print(z.shape)
     #   z+=aft
        return {'z':z}
    def forward(self, g, feature):
  #      print(g)
        g.ndata['h'] = feature
        g.update_all(self.message_func,self.reduce_func)
        g.apply_nodes(func=self.apply_mod)

        return g.ndata.pop('z')
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation
        nn.init.xavier_normal_(self.linear.weight,gain=1.414)

    def forward(self, node):
        z= self.linear(node.data['z'])
        if self.activation is not None:
            z = self.activation(z)
    #    z=F.dropout(z, p=0.5, training=self.training)    
        return {'z' : z}

In [5]:
class Net(nn.Module):
    def __init__(self,in_feat,out_feat,activation):
        super(Net,self).__init__()
        self.gcn1=GCN1(in_feat,out_feat,activation)
        self.gcn2=GCN1(in_feat,out_feat,activation)
        self.gcn3=GCN1(in_feat,out_feat,None)
        self.l1=nn.Linear(in_feat,out_feat)
        self.self_weight = nn.Parameter(torch.FloatTensor(in_feat, out_feat))
        self.reset_parameters()
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.self_weight.size(1))
        self.self_weight.data.uniform_(-stdv, stdv)    
       # self.gcn4=GCN2(in_feat,out_feat,activation)
       # self.gcn5=GCN2(in_feat,out_feat,None)
    def forward(self,g,feature):
        x=self.gcn1(g,feature)
       # y=self.gcn2(g,x)
        y=torch.mm(feature,self.self_weight)
        x+=y
        return x

In [7]:
class MAGNN_metapath_specific(nn.Module):
    def __init__(self,
                 etypes,
                 out_dim,
                 num_heads,
                 attn_drop=0.5,
                 alpha=0.01,
                 use_minibatch=False,
                 attn_switch=False):
        super(MAGNN_metapath_specific, self).__init__()
        self.out_dim = out_dim
        self.num_heads = num_heads
       # self.rnn_type = rnn_type
        self.etypes = etypes
        self.use_minibatch = use_minibatch
        self.attn_switch = attn_switch
        self.coder1=edge_coder(out_dim*3,out_dim,out_dim,0.5)
        self.coder2=edge_coder(out_dim*5,out_dim,out_dim,0.5)
        self.coder3=edge_coder(out_dim*4,out_dim,out_dim,0.5)
        self.coder4=edge_coder(out_dim*3,out_dim,out_dim,0.5)
        self.coder5=edge_coder(out_dim*3,out_dim,out_dim,0.5)
        self.coder6=edge_coder(out_dim*4,out_dim,out_dim,0.5)
        self.Net=Net(out_dim,out_dim,F.relu)
       # self.Multi_Net=Multi_Net(out_dim,out_dim,out_dim)
        # rnn-like metapath instance aggregator
        # consider multiple attention heads
        # node-level attention
        # attention considers the center node embedding or not
        if self.attn_switch:
            self.attn1 = nn.Linear(out_dim, num_heads, bias=False)
            self.attn2 = nn.Parameter(torch.empty(size=(1, num_heads, out_dim)))
        else:
            self.attn = nn.Parameter(torch.empty(size=(1, num_heads, out_dim)))
        self.leaky_relu = nn.LeakyReLU(alpha)
        self.softmax = edge_softmax
        if attn_drop:
            self.attn_drop = nn.Dropout(attn_drop)
        else:
            self.attn_drop = lambda x: x

        # weight initialization
        if self.attn_switch:
            nn.init.xavier_normal_(self.attn1.weight, gain=1.414)
            nn.init.xavier_normal_(self.attn2.data, gain=1.414)
        else:
            nn.init.xavier_normal_(self.attn.data, gain=1.414)

    def edge_softmax(self, g):
        attention = self.softmax(g, g.edata.pop('a'))
        # Dropout attention scores and save them
        g.edata['a_drop'] = self.attn_drop(attention)

    def message_passing(self, edges):
        ft = edges.data['aft'] * edges.data['a_drop']
        return {'ft': ft}
    def message_pass(self,edges):
        return {'h':edges.src['h'],'a':edges.data['a']}
    def reduce_func(self,nodes):
        alpha=F.softmax(nodes.mailbox['a'],dim=1)
        alpha_drop=self.attn_drop(alpha)
        z=torch.sum(alpha_drop*nodes.mailbox['h'],dim=1)
        return {'z':z}
    def forward(self, inputs):
        # features: num_all_nodes x out_dim
        if self.use_minibatch:
            g, features, type_mask, edge_metapath_indices, target_idx,nodes,num = inputs
        else:
            g, features, type_mask, edge_metapath_indices,nodes,num= inputs
     #   print(nodes)
   #     print(g)
     #   print(len(nodes))
        g_feat=F.embedding(nodes,features)
      #  print(g_feat.shape)
    #    g.ndata['h']=g_feat
    #    print(g_feat)
        # Embedding layer
        # use torch.nn.functional.embedding or torch.embedding here
        # do not use torch.nn.embedding
        # edata: E x Seq x out_dim
        
        edata = F.embedding(edge_metapath_indices, features)
       # print(edata.shape)
        
        fdata=edata.reshape(edata.shape[0],-1)
     #   print(fdata.shape)
        if(num==1):
            fdata=self.coder1(fdata)
        if(num==2):
            fdata=self.coder2(fdata)
        if(num==3):
            fdata=self.coder3(fdata)
        if(num==4):
            fdata=self.coder4(fdata)
        if(num==5):
            fdata=self.coder5(fdata)
        if(num==6):
            fdata=self.coder6(fdata)
        
        #eft = hidden.permute(1, 0, 2).view(-1, self.num_heads, self.out_dim)  # E x num_heads x out_dim
   #     print(self.attn.shape)
    #    print(fdata.shape)
        aft=fdata.unsqueeze(1)
   #     print(aft.shape)
        a = (aft * self.attn).sum(dim=-1)
       
       # a = (aft * self.attn).sum(dim=-1).unsqueeze(dim=-1)  # E x num_heads x 1
   #     print('ashape')
   #     print(a.shape)    
     #   a = self.leaky_relu(a)
        aft=aft.squeeze(dim=1)
    #    print(aft.shape)
        g.edata.update({'aft': aft, 'a': a})
        # compute softmax normalized attention values
        self.edge_softmax(g)
        # compute the aggregated node features scaled by the dropped,
        # unnormalized attention values.
        #g.update_all(self.message_passing, fn.sum('ft', 'ft'))
 #       g.update_all(self.message_pass,self.reduce_func)
  #      ret=g.ndata.pop('z')
   #     ret = g.ndata.pop('ft')  # E x num_heads x out_dim
        ret=self.Net(g,g_feat)
        if self.use_minibatch:
            return ret[target_idx]
        else:
            return ret
